In [1]:
# Library untuk pengolahan data dan visualisasi
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import joblib
import threading

# Library untuk evaluasi dan model machine learning
from sklearn.metrics import confusion_matrix, classification_report
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import sklearn.ensemble as ek
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Library untuk Explainable AI (XAI)
from lime.lime_tabular import LimeTabularExplainer
import shap

# Library untuk Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

In [2]:
# Input Dataset 

DM = pd.read_parquet("C:\\Data Raihan\\Penelitian Threshold\\Dataset\\CCCS-CIC-AndMal-2020\\cicandmal2020-dynamic.parquet")

In [3]:
DM.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53439 entries, 0 to 53438
Data columns (total 145 columns):
 #    Column                                                                              Non-Null Count  Dtype 
---   ------                                                                              --------------  ----- 
 0    Memory_PssTotal                                                                     53439 non-null  int32 
 1    Memory_PssClean                                                                     53439 non-null  int32 
 2    Memory_SharedDirty                                                                  53439 non-null  int32 
 3    Memory_PrivateDirty                                                                 53439 non-null  int32 
 4    Memory_SharedClean                                                                  53439 non-null  int32 
 5    Memory_PrivateClean                                                                 53439 non

In [4]:
# Menampilkan nilai unik pada kolom Category, Family, dan Label
unique_category = DM['Category'].unique()
unique_family = DM['Family'].unique()
unique_label = DM['Label'].unique()

print("Unique Categories:", unique_category)
print("Unique Families:", unique_family)
print("Unique Labels:", unique_label)

Unique Categories: ['Trojan_Spy' 'FileInfector' 'Zero_Day' 'Backdoor' 'Ransomware'
 'No_Category' 'Trojan_SMS' 'Trojan_Dropper' 'Trojan_Banker' 'Scareware'
 'PUA' 'Adware' 'Riskware' 'Trojan']
Unique Families: ['smsthief' 'sandr' 'smforw' ... 'pesabti' 'mycompany' 'koomer']
Unique Labels: ['Trojan_Spy_before_reboot_Cat' 'FileInfector_after_reboot_Cat'
 'Zero_Day_after_reboot_Cat' 'Trojan_Spy_after_reboot_Cat'
 'Backdoor_before_reboot_Cat' 'Ransomware_after_reboot_Cat'
 'Ransomware_before_reboot_Cat' 'No_Category_before_reboot_Cat'
 'Trojan_SMS_after_reboot_Cat' 'Zero_Day_before_reboot_Cat'
 'Backdoor_after_reboot_Cat' 'Trojan_Dropper_after_reboot_Cat'
 'Trojan_Banker_after_reboot_Cat' 'Scareware_before_reboot_Cat'
 'PUA_after_reboot_Cat' 'Scareware_after_reboot_Cat'
 'PUA_before_reboot_Cat' 'Adware_before_reboot_Cat'
 'Riskware_before_reboot_Cat' 'Riskware_after_reboot_Cat'
 'Trojan_Banker_before_reboot_Cat' 'Trojan_SMS_before_reboot_Cat'
 'FileInfector_before_reboot_Cat' 'Trojan_befor

In [5]:
# Definisikan kategori benign dan malware
benign_categories = ['Adware', 'Riskware', 'PUA', 'No_Category']
malware_categories = ['Trojan_Spy', 'FileInfector', 'Zero_Day', 'Backdoor', 
                      'Ransomware', 'Trojan_SMS', 'Trojan_Dropper', 'Trojan_Banker', 
                      'Scareware', 'Trojan']

# Relabeling ke dalam kelas binari: 'Benign' dan 'Malware'
DM['Binary_Label'] = DM['Category'].apply(lambda x: 'Benign' if x in benign_categories else 'Malware')

# Cek hasil unique Binary_Label setelah relabeling
unique_binary_labels = DM['Binary_Label'].unique()
print("Unique Binary Labels after relabeling:", unique_binary_labels)

Unique Binary Labels after relabeling: ['Malware' 'Benign']


In [6]:
#Feature Selection
features_to_drop = ['Hash', 'Category', 'Family', 'Label', 'Network_TotalReceivedBytes', 'Binary_Label']

# Droping specified columns and target variable
X = DM.drop(features_to_drop, axis=1).values    
y = DM['Binary_Label'].values

In [7]:
#Remove Nan
X = pd.DataFrame(X).dropna()
y = y[X.index]

In [8]:
# Applying Min-Max scaling to make X non-negative
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
# Initialize RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, n_jobs=-1, class_weight='balanced', random_state=42)

In [10]:
# Boruta Algorithm (Wrapper)
boruta_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=42)
boruta_selector.fit(X_scaled, y)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	140
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	140
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	140
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	140
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	140
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	140
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	140
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	67
Tentative: 	9
Rejected: 	64
Iteration: 	9 / 100
Confirmed: 	67
Tentative: 	9
Rejected: 	64
Iteration: 	10 / 100
Confirmed: 	67
Tentative: 	9
Rejected: 	64
Iteration: 	11 / 100
Confirmed: 	67
Tentative: 	9
Rejected: 	64
Iteration: 	12 / 100
Confirmed: 	69
Tentative: 	7
Rejected: 	64
Iteration: 	13 / 100
Confirmed: 	69
Tentative: 	7
Rejected: 	64
Iteration: 	14 / 100
Confirmed: 	69
Tentative: 	7
Rejected: 	64
Iteration: 	15 / 100
Confirmed: 	69
Tentative: 	7
Rejected: 	64
Iteration: 	16 / 100
Confirmed: 	69
Tentative: 	7

BorutaPy(estimator=RandomForestClassifier(class_weight='balanced',
                                          n_estimators=121, n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x14DC251C940),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x14DC251C940, verbose=2)

In [11]:
# Get selected features
boruta_selected_features = np.where(boruta_selector.support_)[0]

# Extracting column names for selected features
filtered_columns = DM.drop(features_to_drop, axis=1).columns
features = filtered_columns[boruta_selected_features]

# Display the selected features
print("Selected Features from Boruta Algorithm:")
for feature in features:
    print(feature)

Selected Features from Boruta Algorithm:
Memory_PssTotal
Memory_PssClean
Memory_SharedDirty
Memory_PrivateDirty
Memory_SharedClean
Memory_PrivateClean
Memory_HeapSize
Memory_HeapAlloc
Memory_HeapFree
Memory_Views
Memory_ViewRootImpl
Memory_AppContexts
Memory_LocalBinders
Memory_ProxyBinders
Memory_ParcelMemory
Memory_ParcelCount
API_Command_java.lang.Runtime_exec
API_Command_java.lang.ProcessBuilder_start
API_WebView_android.webkit.WebView_loadUrl
API_FileIO_libcore.io.IoBridge_open
API_FileIO_android.content.ContextWrapper_openFileInput
API_FileIO_android.content.ContextWrapper_openFileOutput
API_FileIO_android.content.ContextWrapper_deleteFile
API_Database_android.database.sqlite.SQLiteDatabase_getPath
API_Database_android.database.sqlite.SQLiteDatabase_openDatabase
API_Database_android.database.sqlite.SQLiteDatabase_rawQuery
API_Database_android.database.sqlite.SQLiteDatabase_rawQueryWithFactory
API_Database_android.database.sqlite.SQLiteDatabase_update
API_Database_android.database

In [12]:
# Tentukan 1% dari total dataset
total_samples = int(len(DM) * 0.01)

# Ambil sampel secara acak dari kedua kelas dengan jumlah yang seimbang
sampled_data = DM.groupby('Binary_Label').apply(lambda x: x.sample(n=int(total_samples / 2), random_state=42)).reset_index(drop=True)

# Cek distribusi kelas setelah sampling
print(sampled_data['Binary_Label'].value_counts())

Binary_Label
Benign     267
Malware    267
Name: count, dtype: int64


C:\Users\Muhammad Raihan\AppData\Local\Temp\ipykernel_18592\2187110322.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_data = DM.groupby('Binary_Label').apply(lambda x: x.sample(n=int(total_samples / 2), random_state=42)).reset_index(drop=True)


In [13]:
# Pisahkan data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

In [14]:
# Encode labels ke bentuk numerik jika diperlukan
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [15]:
# Inisialisasi variabel untuk menyimpan hasil evaluasi
hasil_ml_dl = []
hasil_ml_dl_xai = []

# Encode labels ke bentuk numerik jika diperlukan
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Fungsi untuk mengevaluasi model ML/DL
def EvaluateModel(model_name, model, X_train, y_train, X_test, y_test, use_xai=False, is_dl_model=False):
    start_time = time.time()

    # Melatih model
    model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0) if is_dl_model else model.fit(X_train, y_train)
    
    if is_dl_model:
        y_pred_proba = model.predict(X_test)
        y_pred = (y_pred_proba > 0.5).astype(int).ravel()
    elif hasattr(model, 'predict_proba'):
        # Model dengan metode predict_proba
        y_pred_proba = model.predict_proba(X_test)
        if y_pred_proba.shape[1] > 1:  # Model klasifikasi multi-kelas
            y_pred = np.argmax(y_pred_proba, axis=1)
        else:  # Model klasifikasi biner
            y_pred = (y_pred_proba > 0.5).astype(int).ravel()
    else:
        # Model tanpa metode predict_proba
        y_pred_proba = model.predict(X_test)
        y_pred = (y_pred_proba > 0.5).astype(int).ravel()

    # Menghitung confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    # Menghitung metrik
    Precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    Recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    F1Score = 2 * ((Precision * Recall) / (Precision + Recall)) if (Precision + Recall) != 0 else 0
    Accuracy = (tp + tn) / (tp + fp + tn + fn) if (tp + fp + tn + fn) != 0 else 0

    # Menghitung waktu running
    run_time = time.time() - start_time

    # Jika XAI diperlukan, tambahkan analisis dengan SHAP
    if use_xai:
        # Periksa apakah X_train adalah DataFrame
        if isinstance(X_train, pd.DataFrame):
            feature_names = X_train.columns
        else:
            feature_names = [f"Feature_{i}" for i in range(X_train.shape[1])]
        
        # Perbaiki format X_train untuk SHAP
        if is_dl_model:
            X_train_for_xai = X_train.reshape((X_train.shape[0], X_train.shape[1]))
            X_test_for_xai = X_test.reshape((X_test.shape[0], X_test.shape[1]))
        else:
            X_train_for_xai = X_train
            X_test_for_xai = X_test

        # Gunakan SHAP
        try:
            if is_dl_model:
                explainer = shap.DeepExplainer(model, X_train_for_xai)
                shap_values = explainer.shap_values(X_test_for_xai)
                shap_summary = np.mean(shap_values[0], axis=0)
            else:
                explainer = shap.Explainer(model.predict_proba, X_train_for_xai)
                shap_values = explainer(X_test_for_xai)
                shap_summary = shap_values.values.mean(axis=0)
        except Exception as e:
            print(f"Error using SHAP with {model_name}: {e}")
            shap_summary = None

        # Simpan hasil evaluasi dengan XAI
        hasil_ml_dl_xai.append([model_name, Precision, Recall, F1Score, Accuracy, run_time, {'SHAP': shap_summary}])
    else:
        # Simpan hasil evaluasi tanpa XAI
        hasil_ml_dl.append([model_name, Precision, Recall, F1Score, Accuracy, run_time])

# Model ML dan DL yang akan dievaluasi
model_ml_dl = {
    "DecisionTree": DecisionTreeClassifier(max_depth=10),
    "RandomForest": RandomForestClassifier(n_estimators=50),
    "Logistic Regression": LogisticRegression(random_state=0, max_iter=10000),
    "Naive Bayes": GaussianNB(),
    "MLP": MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1, max_iter=1000),
    "Stochastic Gradient Descent": SGDClassifier(loss='log_loss', random_state=42),
    "ADA Boost": AdaBoostClassifier(n_estimators=100),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100),
    "XGBoost": XGBClassifier(n_estimators=100),
    "LightGBM": LGBMClassifier(n_estimators=100),
    "CatBoost": CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='Logloss')
}

model_dl = {
    "DNN": Sequential([
        Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ]),
    "CNN": Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ]),
    "RNN": Sequential([
        LSTM(100, input_shape=(X_train.shape[1], 1)),
        Dense(1, activation='sigmoid')
    ]),
}

# Pastikan X_train dan X_test memiliki bentuk yang sesuai untuk DL
X_train_dl = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_dl = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Mengevaluasi model ML tanpa XAI
for model_name, model in model_ml_dl.items():
    EvaluateModel(model_name, model, X_train, y_train_encoded, X_test, y_test_encoded, use_xai=False)

# Mengevaluasi model DL tanpa XAI
for model_name, model in model_dl.items():
    model.compile(optimizer='adam', loss=BinaryCrossentropy(), metrics=['accuracy'])
    EvaluateModel(model_name, model, X_train_dl, y_train_encoded, X_test_dl, y_test_encoded, use_xai=False, is_dl_model=True)

# Mengevaluasi model ML dengan XAI
for model_name, model in model_ml_dl.items():
    EvaluateModel(model_name, model, X_train, y_train_encoded, X_test, y_test_encoded, use_xai=True)

# Mengevaluasi model DL dengan XAI
for model_name, model in model_dl.items():
    model.compile(optimizer='adam', loss=BinaryCrossentropy(), metrics=['accuracy'])
    EvaluateModel(model_name, model, X_train_dl, y_train_encoded, X_test_dl, y_test_encoded, use_xai=True, is_dl_model=True)

# Print hasil evaluasi tanpa XAI
print("\nHasil Evaluasi ML/DL tanpa XAI:")
print(hasil_ml_dl)

# Print hasil evaluasi dengan XAI
print("\nHasil Evaluasi ML/DL dengan XAI:")
print(hasil_ml_dl_xai)

# Convert results to DataFrame and save to CSV
df_ml_dl = pd.DataFrame(hasil_ml_dl, columns=["Model", "Precision", "Recall", "F1Score", "Accuracy", "RunTime"])
df_ml_dl_xai = pd.DataFrame(hasil_ml_dl_xai, columns=["Model", "Precision", "Recall", "F1Score", "Accuracy", "RunTime", "XAI"])

df_ml_dl.to_csv("hasil_evaluasi_ml_dl_BAFC.csv", index=False)
df_ml_dl_xai.to_csv("hasil_evaluasi_ml_dl_xai_BAFC.csv", index=False)

[LightGBM] [Info] Number of positive: 17620, number of negative: 19787
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.047158 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14604
[LightGBM] [Info] Number of data points in the train set: 37407, number of used features: 120
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.471035 -> initscore=-0.115991
[LightGBM] [Info] Start training from score -0.115991
0:	learn: 0.6420239	total: 207ms	remaining: 20.5s
1:	learn: 0.5995319	total: 268ms	remaining: 13.1s
2:	learn: 0.5669839	total: 317ms	remaining: 10.2s
3:	learn: 0.5398208	total: 363ms	remaining: 8.71s
4:	learn: 0.5198500	total: 432ms	remaining: 8.2s
5:	learn: 0.5038048	total: 501ms	remaining: 7.85s
6:	learn: 0.4879654	total: 552ms	remaining: 7.33s
7:	learn: 0.4774026	total: 620ms	remaining: 7.13s
8:	learn: 0.4653487	total: 665ms	remaining: 6.72s
9:	learn: 0.4549761	total: 736ms	remaining: 6.62s
10:	l

PermutationExplainer explainer: 16033it [18:44, 14.22it/s]                                                             
PermutationExplainer explainer: 16033it [2:24:36,  1.84it/s]                                                           
PermutationExplainer explainer: 16033it [31:39,  8.40it/s]                                                             
PermutationExplainer explainer: 16033it [1:23:32,  3.19it/s]                                                           
PermutationExplainer explainer: 16033it [6:15:23,  1.41s/it]                                                           
PermutationExplainer explainer: 16033it [23:36, 11.24it/s]                                                             
PermutationExplainer explainer: 16033it [18:18:42,  4.11s/it]                                                          
PermutationExplainer explainer: 16033it [1:33:20,  2.86it/s]                                                           
PermutationExplainer explainer: 16033it 

[LightGBM] [Info] Number of positive: 17620, number of negative: 19787
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.997398 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14604
[LightGBM] [Info] Number of data points in the train set: 37407, number of used features: 120
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.471035 -> initscore=-0.115991
[LightGBM] [Info] Start training from score -0.115991


PermutationExplainer explainer: 16033it [1:16:32,  3.48it/s]                                                           


0:	learn: 0.6420239	total: 160ms	remaining: 15.9s
1:	learn: 0.5995319	total: 254ms	remaining: 12.5s
2:	learn: 0.5669839	total: 768ms	remaining: 24.8s
3:	learn: 0.5398208	total: 1.25s	remaining: 29.9s
4:	learn: 0.5198500	total: 1.97s	remaining: 37.4s
5:	learn: 0.5038048	total: 2.52s	remaining: 39.5s
6:	learn: 0.4879654	total: 3.29s	remaining: 43.7s
7:	learn: 0.4774026	total: 4s	remaining: 46s
8:	learn: 0.4653487	total: 4.97s	remaining: 50.2s
9:	learn: 0.4549761	total: 5.43s	remaining: 48.9s
10:	learn: 0.4432840	total: 6.23s	remaining: 50.4s
11:	learn: 0.4322878	total: 6.79s	remaining: 49.8s
12:	learn: 0.4249601	total: 7.29s	remaining: 48.8s
13:	learn: 0.4183419	total: 8.04s	remaining: 49.4s
14:	learn: 0.4113348	total: 8.62s	remaining: 48.9s
15:	learn: 0.4045004	total: 9.21s	remaining: 48.4s
16:	learn: 0.3995319	total: 9.57s	remaining: 46.7s
17:	learn: 0.3958343	total: 9.82s	remaining: 44.8s
18:	learn: 0.3912057	total: 9.94s	remaining: 42.4s
19:	learn: 0.3864993	total: 10.1s	remaining: 4

...

501/501 [==============================] - 5s 9ms/step


C:\Users\Muhammad Raihan\AppData\Local\Programs\Python\Python311\Lib\site-packages\shap\explainers\_deep\deep_tf.py:99: UserWarning: Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
  warnings.warn("Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.")
C:\Users\Muhammad Raihan\AppData\Local\Programs\Python\Python311\Lib\site-packages\shap\explainers\_deep\deep_tf.py:155: UserWarning: You have provided over 5k background samples! For better performance consider using smaller random sample.
  warnings.warn("You have provided over 5k background samples! For better performance consider using smaller random sample.")
C:\Users\Muhammad Raihan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\backend.py:452: UserWarning: `tf.keras.backend.s

Error using SHAP with CNN: operands could not be broadcast together with shapes (37407,140,1) (37407,140) 
501/501 [==============================] - 22s 40ms/step


C:\Users\Muhammad Raihan\AppData\Local\Programs\Python\Python311\Lib\site-packages\shap\explainers\_deep\deep_tf.py:99: UserWarning: Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
  warnings.warn("Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.")
C:\Users\Muhammad Raihan\AppData\Local\Programs\Python\Python311\Lib\site-packages\shap\explainers\_deep\deep_tf.py:155: UserWarning: You have provided over 5k background samples! For better performance consider using smaller random sample.
  warnings.warn("You have provided over 5k background samples! For better performance consider using smaller random sample.")
C:\Users\Muhammad Raihan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\backend.py:452: UserWarning: `tf.keras.backend.s

Error using SHAP with RNN: in user code:

    File "C:\Users\Muhammad Raihan\AppData\Local\Programs\Python\Python311\Lib\site-packages\shap\explainers\_deep\deep_tf.py", line 244, in grad_graph  *
        out = self.model(shap_rAnD)
    File "C:\Users\Muhammad Raihan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Muhammad Raihan\AppData\Local\Programs\Python\Python311\Lib\site-packages\shap\explainers\_deep\deep_tf.py", line 371, in custom_grad
        out = op_handlers[type_name](self, op, *grads) # we cut off the shap_ prefix before the lookup
    File "C:\Users\Muhammad Raihan\AppData\Local\Programs\Python\Python311\Lib\site-packages\shap\explainers\_deep\deep_tf.py", line 663, in handler
        return linearity_with_excluded_handler(input_inds, explainer, op, *grads)
    File "C:\Users\Muhammad Raihan\AppData\Local\Programs\Python\P